# Assess MDPO Results
Loads MDPO pickle file and evaluates training over time.
Also equipped to perform SEPIO on device placements to determine the limits of classificaiton accuracy.

In [ ]:
### Imports and Files ###
import matplotlib.pyplot as plt
import numpy as np
from os import path
import sys
import pickle
sys.path.insert(0, path.join('../..'))
from modules.leadfield_importer import FieldImporter
from scipy.io import loadmat
from modules.SEPIO import sepio

Provides graphing of MDPO results and performs SEPIO on desired sensor sets

## Finding device locations
Settings __MUST__ match those chosen prior in optimization!

Can potentially switch devices __IF__ they are the same shape lead fields (excluding sensor axis).

In [ ]:
# Brainsuite files
# List all components of the 'full brain' and 'ROI', where ROI is weighted for each
folder = r"...\SEPIO_dataset" # Point to dataset folder

# Auditory cortex test
brain_data = [loadmat(path.join(folder,'MDPO_data','brain.mat'))]
roi_data = [loadmat(path.join(folder,'MDPO_data','auditory_roi.mat'))]
roi_weights = [1]

do_weights = False

# Define dipoles
# Magnitude is 0.5e-9 (nAm) for real signals or 20e-9 (nAm) for phantom simulations
magnitude = 0.5e-9 # nAm; Default power at given cortical column length
dipole_base_length = 2.5 # mm; Default cortical column length

# Load lead fields
fields_files = path.join(folder,'leadfields',"SEEG_8e_500um_1500pitch.npz")
#fields_files = path.join(folder,'leadfields',"DISC_30mm_p2-5Sm_MacChr.npz")
#fields_files = path.join(folder,'leadfields',"ECoG_1mm_500umgrid.npz")

# DiSc specific parameters
macros = False # Bool; Reduce DiSc to virtual macros
virtual_rings = 4 # Must be one of [1,2,4,8]

# Processing parameters
sensor_div = 8 # Sensor step size for SEPIO; MUST SATISFY -> sensor_max % sensor_div = 0
                # 16 for DiSc; 2/4 for SEEG (8, 16, 18 sensors)
scale = 0.5 # mm; voxel size
cl_wd = 1.0 # mm diameter to clear; device may be slightly offset
noise = 4.1 # uV RMS noise; 4.1 for DiSc; 2.7 for SEEG; Varing for ECoG based on diameter
bandwidth = 100 # Samples/second; Recording system limit or reasonable maximum
Montage = False # Montage devices
field_importer = FieldImporter()
field = field_importer.load(fields_files)
num_electrodes = np.shape(field_importer.fields)[4]
fields = field_importer.fields
midpoint = fields.shape[0]//2  # field midpoint index

In [ ]:
# GA file locations
out = path.join(folder,"outputs")

# Auditory cortex used for paper
files = [r"20241024_1054-genetic-1.pkl",r"20241024_1115-genetic-2.pkl",r"20241024_1146-genetic-3.pkl",
         r"20241024_1227-genetic-4.pkl",r"20241024_1313-genetic-5.pkl"]

# Assign file N
N_file = np.arange(1,len(files)+1) # Number of devices used for each file; INT only; Could be made automatic from file contents
pickle_files = []
for f in files:
    pickle_files.append(path.join(folder,'MDPO_data',f))

# Load Pickle file
data = []
for p in pickle_files:
    with open(p,'rb') as f:
        data.append(pickle.load(f))

# Assess data dict
''' KEYS
'type'
'best solution:'
'best solution fitness'
'fitnesses'
'solutions'
'device names'
'device types'
'field files'
'N per device'
'LF scale'
'dev diameter'
'dev depth'
'dev bacend'
'voltage scale'
'dev bandwidth'
'depth limits'
'angle limits'
'montaged'
''' 

best = []
radians = False # Return radians or degrees; MUST be FALSE for SEPIO; MUST be TRUE for visualization plotting
if radians:
    print("!!!DISABLE `radians` to use SEPIO!!!")
print('Data')
for i,f in enumerate(files):
    print("N =",N_file[i])
    depth = sum(isinstance(v, dict) for v in data[i].values()) # counts sub-dicts
    bredth = len(next(iter(data[i].values()))) # counts dict bredth
    print('Depth:',depth,'\nBredth',bredth)
    for key, value in data[i].items():
        print('key',key)#, value) # Prints all keys and values
    best.append(data[i]['best solution:']) # Degrees
    
    # Convert to degrees
    if radians:
        for j in np.arange(N_file[i]): # j for each device
            best[i][6*j+3] = best[i][6*j+3]*180/np.pi
            best[i][6*j+4] = best[i][6*j+4]*180/np.pi
            best[i][6*j+5] = best[i][6*j+5]*180/np.pi
    print(f"File #{i+1}:")
    print(best[i].reshape(best[i].shape[0]//6,6))

NOTE: 'best solution' and 'best solution fitness' are swapped in early anneal and gradient methods

## Plotting fitness

In [ ]:
data = data[0]
best_solution = data['best solution:']
best_solution_fitness = data['best solution fitness']
fitnesses = data['fitnesses']
best_sols = data['best solution over generations']
mean_fitnesses = data['mean fitnesses']
std_fitnesses = data['std fitnesses']

N = np.array(data['best solution:']).shape[0]//6
num_generations = len(data['fitnesses']) # Adjusts to loaded data
generations = list(np.arange(0,num_generations,1))

for gen in generations:
    best_sol = best_sols[gen].reshape((N, 6))

In [ ]:
### Graph best fitness scores versus device count ###
fitness = np.empty((len(N_file),))
for i in range(len(N_file)): # i for each file
    fitness[i] = data[i]['best solution fitness']

plt.plot(N_file,fitness,label="Total IC")
for i,f in enumerate(fitness):
    fitness[i] = f/N_file[i]
plt.plot(N_file,fitness,label="IC per device")
#plt.yscale('log')
plt.xticks(N_file)
plt.ylabel('Information Capacity (Bits/s.)')
plt.xlabel('Number of Devices')
plt.title(f"Fitness per device\n({data[0]['type'][0]})")
plt.legend(loc='right')
plt.savefig(path.join(out,f"8_best_fitness-{data[0]['type'][0]}.png"),dpi=300)
plt.show()

In [ ]:
### Graph the fitness gain over generations for all device counts ###
if data[0]['type'][0] == 'anneal':
    # Make graph relative to heat, not generations
    fitness = []
    heat = []
    for i in np.arange(len(N_file)): # i for each file
        f = []
        h = []
        for k in np.arange(len(data[i]['fitnesses'])): # k for each fitness/heat per file
            f.append(data[i]['fitnesses'][k])
            h.append(data[i]['temperature'][k])
        fitness.append(f)
        heat.append(h)

    for i,n in enumerate(N_file):
        if n == 1:
            plt.plot(heat[i],fitness[i],label=f'{n} Device')
        else:
            plt.plot(heat[i],fitness[i],label=f'{n} Devices')
    #plt.yscale('log')
    plt.ylabel('Information Capacity (Bits/s.)')
    plt.xlabel('Temperature')
    plt.xscale('log')
    plt.gca().invert_xaxis()
    plt.title(f"Fitness per device over annealing temp.\n({data[0]['type'][0]})")
    plt.legend(loc='lower right')
    plt.savefig(path.join(out,f"8_fitness_over_generations-{data[0]['type'][0]}.png"),dpi=300)
    plt.show()
else:
    num_generations = len(data[0]['fitnesses'])
    generations = list(np.arange(0,num_generations,1))
    fitness = np.zeros((len(N_file),num_generations))
    for i in np.arange(len(N_file)): # i for each file
        fitness[i] += data[i]['fitnesses']

    for i,n in enumerate(N_file):
        if n == 1:
            plt.plot(generations,fitness[i],label=str(n)+' Device')
        else:
            plt.plot(generations,fitness[i],label=str(n)+' Devices')
    #plt.yscale('log')
    plt.ylabel('Information Capacity (Bits/s.)')
    plt.xlabel('Generations')
    plt.title(f"Fitness per device over generations\n({data[0]['type'][0]})")
    plt.legend(loc='lower right')
    plt.savefig(path.join(out,f"8_fitness_over_generations-{data[0]['type'][0]}.png"),dpi=300)
    plt.show()

## Re-generating voltages and performing SEPIO

### Functions

In [ ]:
def virtual_macros(field,rings):
    # Reduce a DiSc LF into virtual macros
    # 'rings' defines how many macros to divide into (1,2,4,or 8)
    rings = int(rings)
    if field.shape[-1] == 64: # 64 ch DiSc
        rows_per_ring = 8//rings
        e_per_ring = rows_per_ring*8
        new_field = np.zeros((field.shape[0],field.shape[1],field.shape[2],3,rings))
        for r in range(rings): # r for each ring; top down
            for c in range(8): # c for each column
                for d in range(rows_per_ring): # d for column depth in each ring
                    new_field[:,:,:,:,r] += field[:,:,:,:,c*8+r*rows_per_ring+d]
        new_field /= e_per_ring
        return new_field
    else:
        print("Incorrect input LF shape.")
        return field

def field_reduce(field,count):
    if field.shape[-1] == 128: # Starting with 128 channels
        if count == 128:
            field_out = field
        if count == 64:
            keep = np.arange(0,128,2)
            field_out = field[:,:,:,:,keep]
        else:
            print("ERROR: No valid sensor count provided!")
            field_out = None # break process
    else:
        print("ERROR: Original LF not provided. LF unchanged.")
        field_out = field
    return field_out

def obtain_data(data, name):
    
    """
    Extract data from a given file
    
    inputs: 
        - data = data obtained by the function "loadmat".
        - name = a string that is the name of that brain region in the filename saved in the user's computer

    output: header, faces, vertices, and normals from the provided file
    """

    # creating corresponding lists in python in place of matlab variables
    header = data['__header__']
    faces = data[name][0][0][0]
    vertices = data[name][0][0][1]
    normals = data[name][0][0][2]

    return header, faces, vertices, normals


def recenter(vertices, reference):

    """
    Recenter the data
    
    inputs: 
        - vertices: vertices extracted from a specific brain region
        - reference for avg1, avg2, avg3 calculated from the reference brian region
        
    outputs: vertices_modified, the data in vertices, recentered based on avg1, avg2, and avg3
    """

    # find midpoint
    center = np.mean(reference, axis = 0)
    avg1 = center[0]
    avg2 = center[1]
    avg3 = center[2]
    # print(avg1, avg2, avg3)
    
    # create a copy of the vertices
    vertices_modified = vertices.copy()

    # modify the data according to the midpoint
    for idx in range(vertices.shape[0]):
        vertices_modified[idx][0] -= avg1
        vertices_modified[idx][1] -= avg2
        vertices_modified[idx][2] -= avg3

    return vertices_modified

def get_rotmat(alpha, beta, gamma):
    """
    Given device position in standard form, return rotation matrix
    """
    # define rotation matrix 
    yaw = np.array([[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]])
    pitch = np.array([[np.cos(beta), 0, np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]])
    roll = np.array([[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]])
    rot_mat = np.matmul(yaw, pitch)
    rot_mat = np.matmul(rot_mat, roll)

    return rot_mat

def transform_vectorspace(vertices, normals, devpos, max_depth = None):
    """
    Transforms the MRI space depending on the device location in order to calculate voltage
    inputs:
        - vertices, array of vertices wrs to the original MRI coordinates
        - normals, array of normal vectors wrs to the original MRI coordinates
        - devpos, manually defined device position
    output: 
        - dippos, shifted and rotated vertices depending on the device location
        - dipvec, shifted and rotated normal vectors depending on the device location
    """

    rot_mat = get_rotmat(devpos[3],devpos[4],devpos[5])
    inv_rot_mat = np.linalg.inv(rot_mat)

    ###
    ##### No transformation done above this line #####
    ###

    dippos = np.copy(vertices)
    dipvec = np.copy(normals)

    # translation
    dippos-=devpos[:3]
    dippos[0] += fields.shape[0]//2
    dippos[1] += fields.shape[1]//2

    # rotate vertices and normals based on the placement of the device
    for idx in range(vertices.shape[0]):   # rotate for each vertex point and corresponding vectors
        dippos[idx] = np.matmul(inv_rot_mat, dippos[idx])
        dipvec[idx] = np.matmul(inv_rot_mat, dipvec[idx])

    # Transfer position into leadfield space
    dippos *= 1/scale

    # to make it compatible with the lead field data
    dippos = (dippos.astype('int')).astype('float')

    # check if the depth restriction is exceeded
    if max_depth is not None:
        if dippos[2].any() < max_depth:
            ## code to adjust depth
            pass
    
    # Scale to dipole nAm magnitude, see value above
    dipvec *= magnitude 

    return dippos, dipvec, rot_mat

def trim_data(leadfield, vertices, normals):
    """
    Set values outside of the leadfield as nan so that it is compatible for further calculation
    inputs:
        - leadfield, leadfield data in the form of [x,y,z,[vx,vy,vz],e]
        - vertices, transformed data of the vertices (dippos)
        - normals, transformed data of the normal vectors (dipvec)
    outputs:
        - trimmed dippos, dipvec
    """

    dippos = np.copy(vertices)
    dipvec = np.copy(normals)
    # half of side length
    len_half = leadfield.shape[0]//2

    # um = 0
    for idx in range(vertices.shape[0]):
        if((np.abs(vertices[idx][0])>len_half) or (np.abs(vertices[idx][1])>len_half) or (vertices[idx][2])>len_half*2) or (vertices[idx][2]<0):
            # print("yes", um)
            # um+=1
            dippos[idx] = np.nan

    return dippos, dipvec

def calculate_voltage(fields, vertices, normals, vscale=10**6, montage = Montage, inter = False):
    """
    Calculates voltage for each vertex
    input: 
        - fields, leadfield data in the form of [x,y,z,[vx,vy,vz],e]
        - vertices, transformed and trimmed data of the vertices (dippos)
        - normals, transformed and trimmed data of the normal vectors (dipvec)
        - vscale, to scale it to uV
    output: 
        - opt_volt, a 1-D array that has the optimal voltage for each vertex
    """
    global roi_vertices_weights, do_weights
    
    # make copies of data
    dippos = np.copy(vertices)
    dipvec = np.copy(normals)

    # create a list of field vectors applicable to the surface, [vertex index, electrode #]
    voltage = np.empty((dippos.shape[0], fields.shape[-1]))
    voltage[:]=np.nan
    opt_volt = np.empty(dippos.shape[0])
    opt_volt[:]=np.nan
    
    # calculate voltage for each vertex
    for idx in range(dippos.shape[0]): # for each dipole
        for e in range(fields.shape[-1]): # for every electrode
            if not np.any(np.isnan(dippos[idx])): # if not nan, calculate and modify entry
                try:
                    lead_field = fields[int(dippos[idx][0]+fields.shape[0]//2), int(dippos[idx][1]+fields.shape[0]//2), int(dippos[idx][2]), :, e]
                    voltage[idx, e] = np.dot(lead_field, dipvec[idx])
                except: # error in finding lead field
                    voltage[idx, e] = np.nan
        if (not np.all(np.isnan(voltage[idx,:]))) and (not montage): 
            opt_volt[idx] = np.nanmax(np.abs(np.copy(voltage[idx]))) # get the optimal voltage across all electrodes
        elif (not np.all(np.isnan(voltage[idx,:]))) and montage and (not inter):
            single = np.nanmax(np.abs(np.copy(voltage[idx]))) # get the optimal voltage across all electrodes
            mont = np.nanmax(voltage[idx]) - np.nanmin(voltage[idx])
            if mont > single: # Use montage if it's better
                opt_volt[idx] = mont
            else: # Keep zero reference if it's better
                opt_volt[idx] = single
        # elif (not np.all(np.isnan(voltage[idx,:]))) and montage and inter:
    voltage *= vscale # (sources, electrodes) for one device
    opt_volt *= vscale #scale it to get the list of voltages
    snr_list = np.copy(opt_volt)/noise # get the list of snr values
    info_cap = bandwidth*np.log2(1+snr_list) # get the list of information capacity
    snr_list[np.isnan(snr_list)] = 0

        # Scale with given weights for ROI subregions
    if do_weights:
        opt_volt = np.multiply(opt_volt,roi_vertices_weights)
        snr_list = np.multiply(snr_list,roi_vertices_weights)
        info_cap = np.multiply(info_cap,roi_vertices_weights)


    return (opt_volt, snr_list, info_cap, voltage)

def find_snr(devpos,value):
    """
    Find the total SNR generated by N devices.
    value: 0:voltage montaged :: 1:SNR :: 2:IC :: 3 or -1:raw voltage
    """
    devpos = devpos.reshape((N, 6))
    if value == 2: # IC
        all_dev_volt = np.empty((recentered_roi.shape[0], N))
    else:
        all_dev_volt = np.empty((recentered_roi.shape[0], N, fields.shape[-1]))
    for idx in range(len(devpos)):
        dippos, dipvec, rotmat = transform_vectorspace(recentered_roi, roi_normals, devpos[idx])
        dippos_adj, dipvec_adj = trim_data(fields, dippos, dipvec)
        # Index decides volt, snr, IC, or voltage by(source,electrodes)
        if value == 2: # IC
            all_dev_volt[:, idx] = calculate_voltage(fields, dippos_adj, dipvec_adj, montage = Montage, inter = False)[value]
        else:
            all_dev_volt[:, idx,:] = calculate_voltage(fields, dippos_adj, dipvec_adj, montage = Montage, inter = False)[value]
    
    voltage = np.nanmax(np.nan_to_num(all_dev_volt, nan=0), axis=1)
    total = np.nansum(voltage)
    # print("total IC:", total, "Bits/s")
    return all_dev_volt

### Voltage calculations

In [ ]:
### Adjust fields to desired size ###
disc_sensors = 64 # 64 or 128 for DiSc; sensors reduced evenly  in grid
fields = field_reduce(fields,disc_sensors)

In [ ]:
### Calculate virtual macros from DiSc ###
if macros:
    fields = virtual_macros(fields,virtual_rings)

In [ ]:
### Load brain and ROI ###
# Values to collect
brain_vals = []
brain_vallen = [[],[],[]] # summative index for each start point
brain_count = [0,0,0] # running index sum
roi_vals = []
roi_vallen = [[],[],[]]
roi_count = [0,0,0]
# Start indexing counts per file
#for b in brain_data:
#    brain_count.append(0)
#for b in roi_data:
#    roi_count.append(0)

# Load each file
for i,fb in enumerate(brain_data):
    _, brain_faces, brain_vertices, brain_normals = obtain_data(brain_data[i], 'brain') # 'ans' or 'brain'
    brain_vals.append([brain_faces,brain_vertices,brain_normals])
    for k in range(3):
        brain_vallen[k].append(brain_vals[i][k].shape[0] + brain_count[k])
        brain_count[k] += brain_vals[i][k].shape[0]
for i,fb in enumerate(roi_data):
    _, roi_faces, roi_vertices, roi_normals = obtain_data(roi_data[i], 'auditory_roi') # 'ans' or 'auditory roi'
    roi_vals.append([roi_faces,roi_vertices,roi_normals])
    for k in range(3):
        roi_vallen[k].append(roi_vals[i][k].shape[0] + roi_count[k])
        roi_count[k] += roi_vals[i][k].shape[0]

# Initialize final variables
brain_faces = np.empty((brain_vallen[0][-1],3))
brain_vertices = np.empty((brain_vallen[1][-1],3))
brain_normals = np.empty((brain_vallen[2][-1],3))
roi_faces = np.empty((roi_vallen[0][-1],3))
roi_vertices = np.empty((roi_vallen[1][-1],3))
roi_normals = np.empty((roi_vallen[2][-1],3))
roi_vertices_weights = np.zeros((roi_vallen[1][-1],)) # matching index weights

# Merge file variables
for i in range(len(brain_data)): # i for each file index
    if i == 0: # first file starts from zero in lists
        brain_faces[0:brain_vallen[0][i]] = brain_vals[i][0]
        brain_vertices[0:brain_vallen[1][i]] = brain_vals[i][1]
        brain_normals[0:brain_vallen[2][i]] = brain_vals[i][2]
    else:
        brain_faces[brain_vallen[0][i-1]:brain_vallen[0][i]] = brain_vals[i][0]
        brain_vertices[brain_vallen[1][i-1]:brain_vallen[1][i]] = brain_vals[i][1]
        brain_normals[brain_vallen[2][i-1]:brain_vallen[2][i]] = brain_vals[i][2]
for i in range(len(roi_data)): # i for each roi file
    if i == 0: # first file starts from zero in lists
        roi_faces[0:roi_vallen[0][i]] = roi_vals[i][0]
        roi_vertices[0:roi_vallen[1][i]] = roi_vals[i][1]
        roi_normals[0:roi_vallen[2][i]] = roi_vals[i][2]

        roi_vertices_weights[0:roi_vallen[1][i]] = np.repeat(roi_weights[i],roi_vallen[1][i])
    else:
        roi_faces[roi_vallen[0][i-1]:roi_vallen[0][i]] = roi_vals[i][0]
        roi_vertices[roi_vallen[1][i-1]:roi_vallen[1][i]] = roi_vals[i][1]
        roi_normals[roi_vallen[2][i-1]:roi_vallen[2][i]] = roi_vals[i][2]
        roi_vertices_weights[roi_vallen[1][i-1]:roi_vallen[1][i]] += np.repeat(roi_weights[i],(roi_vallen[1][i] - roi_vallen[1][i-1]))

# Recenter total ROI based on total brain
recentered_roi = recenter(roi_vertices, brain_vertices)

In [ ]:
### Calculate voltages for best position set ###
voltages = []  # Must be flexible list for different sensor counts
infocap = [] # Collect IC values
sensor_count = fields.shape[-1]

# Manual trajectory injection
manual = False

# Manual trajectory demo
manual_trajectories = [ # Each inside list is (N*6,) for coordinates for N devices
    np.array([0, 0, 5, 0, 0, 180]),
    np.array([0, 3, 4.5, 0, 0, 180,
              0, -3, 6.5, 0, 0, 180]),
    np.array([-.5, -3, 6, 0, 0, 180,
              -.5, 3, 4.5, 0, 0, 180,
              2, 0, 5, 0, 0, 180])
]

if manual: # Rebuild device count list
    N_file = []
    for t in manual_trajectories:
        N_file.append(t.shape[0]//6)
        if t.shape[0]%6 != 0:
            print("Error: Manual trajectory incorrect.")
            break

# Main loop
for i,N in enumerate(N_file): # each i file of N devices
    if manual:
        voltages.append(find_snr(manual_trajectories[i],-1))
        infocap.append(find_snr(manual_trajectories[i],2))
    else:
        voltages.append(find_snr(best[i],-1))
        infocap.append(find_snr(best[i],2))
    voltages[i] = voltages[i].reshape(-1,N*sensor_count)
    print(f"Done with {N} devices with voltage array shape: {voltages[i].shape}")

### Assess voltage and IC
Not used in paper

In [ ]:
# Sanity check
print(infocap[0].shape)
print(np.count_nonzero(np.isnan(infocap[0])))
print(np.count_nonzero(np.isnan(np.sum(voltages[0],axis=1))))
print(np.where(~np.isnan(infocap[0]))[0])

In [ ]:
### Plot IC distribution for one device by distance of sources ###
# Avoid NaN indices that are not observed by the dataset
notnan = np.where(~np.isnan(infocap[0]))[0] # safe indices

# Distance to each point from the device assigned position; matched indices to voltage
distance = recentered_roi - best[0][0:3]
distance = np.sqrt(np.sum(distance**2,axis=-1))[notnan]

# Plot source distances
plt.hist(distance,bins=50, color='lightblue',edgecolor='black')
plt.xlim([0,30])
plt.xlabel('Device-Source Distance (mm)')
plt.ylabel('Source Count')
plt.savefig(path.join(out,f"8_Distance-distribution.png"),dpi=300)
plt.show()

In [ ]:
# Plot IC voilin for DiSc and vSEEG at varying range
bins = [0,7.5,15,22.5,30] # mm distance groups
binned_ic = [[] for _ in range(len(bins)-1)]
positions = []
for i in (range(len(bins)-1)):
    positions.append((bins[i]+bins[i+1])/2)
positions = np.asarray(positions)

for dist, ic in zip(distance,infocap[0][notnan]):
    bin_idx = np.digitize(dist,bins) - 1
    if 0<=bin_idx<(len(bins)-1):
        binned_ic[bin_idx].append(ic[0])

plt.violinplot(binned_ic,positions=positions,showmeans=True,widths=(bins[1]-bins[0])/1.5)
plt.xticks(bins)
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.xlabel('Binned Distance (mm)')
plt.ylabel('Info. Cap. Distribution')
plt.savefig(path.join(out,f"8_IC-distribution-{voltages[0].shape[-1]}e.png"),dpi=300)
plt.show()

In [ ]:
### Assess voltages ###
print("Voltage summary")
for i in range(len(N_file)):
    print("N:",N_file[i])
    print("Mean:",np.nanmean(voltages[i]))
    #print("Max:",np.nanmax(voltages[i]))
    #print("Min:",np.nanmin(voltages[i]))
    print("STD:",np.nanstd(voltages[i]))

In [ ]:
### Assess Information Capacity ###
print("Information capacity summary")
for i in range(len(N_file)):
    print("N:",N_file[i])
    print("Mean:",np.nanmean(np.nanmax(infocap[i],axis=1)))
    #print("Max:",np.nanmax(np.nanmax(infocap[i],axis=1)))
    #print("Min:",np.nanmin(np.nanmax(infocap[i],axis=1)))
    print("STD:",np.nanstd(np.nanmax(infocap[i],axis=1)))

### SEPIO

In [ ]:
### SEPIO train/test with variable replicates ###
MC_cycles = 1 # How many Monte-Carlo cycles to complete and average
rep_per_max = 1 # (INT) replicates per max sensors
sensor_div = 16 # Sensor testing granularity (defines sensor_range steps)
noise_mult = 10. # Boost noise uniformly
totalcoefs = [] # Coefficients per test set
totalacc = [] # Test acc (sensor space)
spatialacc = [] # Test acc (source space)
sensor_list = [] # list of sensor_range

for i, N in enumerate(N_file):
    print(f"Starting loop {i+1} with {N} devices.")
    MCcoefs, MCaccs, MCSaccs, sensor_range = sepio.mc_train(X=voltages[i],y=None,Xt=None,yt=None,sensor_div=sensor_div,
    MCcount=MC_cycles,noise=noise*noise_mult,replicates=rep_per_max*voltages[i].shape[-1],nbasis=20,spatial=False,l1=0.001,rep_subdiv=8)
    totalcoefs.append(MCcoefs)
    totalacc.append(MCaccs)
    spatialacc.append(MCSaccs)
    sensor_list.append(sensor_range)

In [ ]:
# Check top-N sensors alone
if False:
    topN = 16
    dataset = 0 # index for file
    rep_per_max = 1
    sensor_div = 2 # Sensor testing granularity (defines sensor_range steps)
    sensors = np.flip(np.argsort(np.mean(np.abs(totalcoefs[dataset]),axis=1)))[:16]
    TNcoefs, TNaccs, TNSaccs, TNsensor_range = sepio.mc_train(X=voltages[dataset][:,sensors],y=None,Xt=None,yt=None,sensor_div=sensor_div,
    MCcount=1,noise=4.1,replicates=rep_per_max*voltages[dataset][:,sensors].shape[-1],nbasis=20,spatial=False,l1=0.001,rep_subdiv=8)

In [ ]:
print(np.nanmin(np.array(MCSaccs)))
print(np.nanmean(np.array(MCSaccs)),np.std(np.nan_to_num(np.array(MCSaccs))))
print(np.nanmax(np.array(MCSaccs)))

In [ ]:
print(totalacc)
for n in range(len(totalacc)):
    print(totalacc[n][-1]*100)

In [ ]:
### Plotting accuracies for each device set ###
for i,N in enumerate(N_file):
    if i < len(totalacc):
        if N == 1:
            plt.plot(np.arange(0,sensor_count,sensor_div)+sensor_div,100*totalacc[i],label="1 Device")
        else:
            plt.plot(np.arange(0,sensor_count*N,sensor_div)+sensor_div,100*totalacc[i],label=f"{N} Devices")

plt.ylabel(f"% Accuracy for {voltages[0].shape[0]} classes")
plt.xlabel("Sensor count")
plt.title(f"SEPIO classification accuracy\n({data[0]['type'][0]})")
plt.legend(loc='lower right')
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.savefig(path.join(out,f"8_SEPIO_total-{data[0]['type'][0]}.png"),dpi=300)
plt.show()

In [ ]:
### Plot sensorXdevice for different levels of accuracy ###
# Use totalacc (N, acc[i] for i at sensors*16)

# Assign plot values
xs = []
ys = []
zs = []
for n in range(len(totalacc)): # k is acc
    # n+1 for each device count
    x = []
    y = []
    for s in range(totalacc[n].size):
        # (s+1)*16 for each sensor count
        xs.append((s+1)*sensor_div)
        ys.append(N_file[n])
        zs.append(totalacc[n][s]*100)

# Plot settings
plot_divs = 18 # number of plot divisions
#plt.tricontour(xs,ys,zs,levels=plot_divs,colors='k') # Adds lines between regions
pt = plt.tricontourf(xs,ys,zs,levels=plot_divs,cmap="RdBu_r") # colors in regions

plt.yticks(N_file[:len(totalacc)])
plt.xlim([0,max(N_file[:len(totalacc)])*fields.shape[-1]])
plt.ylabel("Device count")
plt.xlabel("Sensor count")
plt.title(f"SEPIO by device and sensor count\n({data[0]['type'][0]})")
cb = plt.colorbar(pt)
cb.set_label("% Accuracy")
plt.savefig(path.join(out,f"8_SEPIO_SxN-{data[0]['type'][0]}.png"),dpi=300)
plt.show()